In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import numpy as np
import os
import gensim.models.word2vec as w2v
from sklearn.model_selection import train_test_split

D:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train_data = pd.read_csv('C:/Users/jxjsj/Desktop/tianchi/steam/zhengqi_train.txt',sep="\t")
test_data = pd.read_csv('C:/Users/jxjsj/Desktop/tianchi/steam/zhengqi_test.txt',sep="\t")

In [3]:
X = train_data.iloc[:,list(range(len(train_data.columns)-1))]
labels = train_data.iloc[:,len(train_data.columns)-1]

In [70]:
len(test_data)

1925

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(X, labels, random_state=1, train_size=0.8)

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
# 运用原非平衡样本-训练
carEvaluation = MLPRegressor(activation='tanh', 
                    solver='lbfgs',
                    alpha=1e-5,
                    hidden_layer_sizes=(len(x_train.columns)*2+1,),
                    random_state=1, 
                   )
carEvaluation.fit(x_train,y_train)

# 运用原非平衡样本-评估
y_train_predict = carEvaluation.predict(x_train)
y_valid_predict = carEvaluation.predict(x_valid)

print('trainAccracy:',carEvaluation.score(x_train,y_train))
# print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('testAccracy:',carEvaluation.score(x_valid,y_valid))
# print(classification_report(y_test,y_test_predict))

trainAccracy: 0.9731698242811728
testAccracy: 0.8517487608338634


In [33]:
# bpNN模型构造 - BN处理
class bpNN(nn.Module):
    def __init__(self):
        super(bpNN, self).__init__()
        #需要将事先训练好的词向量载入
        self.fc1 = nn.Sequential(
                      nn.Linear(len(x_train.columns), 2*len(x_train.columns)+1),
#                       nn.BatchNorm2d(num_features=1, eps=1e-05, momentum=0.1, affine=True), # BN 处理
                      nn.Sigmoid()
                     )
        self.fc2 = nn.Sequential(
                      nn.Linear(2*len(x_train.columns)+1, 50),
                      nn.BatchNorm2d(num_features=1, eps=1e-05, momentum=0.1, affine=True), # BN 处理
                      nn.Sigmoid()
                     )
        self.out = nn.Linear(50,1)

    def forward(self, x):
#         x = self.embeding(x)
#         x=x.view(x.size(0),1,max_len,word_dim)
        #print(x.size())
        x = self.fc1(x)
        x = self.fc2(x)
        output = self.out(x)
#         conv1_3 = self.conv13(x)
#         x = torch.cat([conv1_1,conv1_2,conv1_3],1)
#         x = x.view(x.size(0), -1) # 将（batch，outchanel,w,h）展平为（batch，outchanel*w*h）
#         output = self.out(x)
        return output

In [36]:
# bpNN模型构造 - BN处理
class bpNN(nn.Module):
    def __init__(self):
        super(bpNN, self).__init__()
        #需要将事先训练好的词向量载入
        self.fc1 = nn.Sequential(
                      nn.Linear(len(x_train.columns), 1)
                     )
#         self.out = nn.Linear(50,1)

    def forward(self, x):
#         x = self.embeding(x)
#         x=x.view(x.size(0),1,max_len,word_dim)
        #print(x.size())
        x = self.fc1(x)
#         x = self.fc2(x)
#         output = self.out(x)
#         conv1_3 = self.conv13(x)
#         x = torch.cat([conv1_1,conv1_2,conv1_3],1)
#         x = x.view(x.size(0), -1) # 将（batch，outchanel,w,h）展平为（batch，outchanel*w*h）
#         output = self.out(x)
        return x

In [37]:
cnn = bpNN()

#构建textCNN模型超参数与数据封装入加载器 - LSC

LR = 0.01

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)

#损失函数
loss_function = nn.MSELoss()

x_train_t = torch.Tensor(np.array(x_train))
y_train_t = torch.Tensor(np.array(y_train))
x_valid_t = torch.Tensor(np.array(x_valid))
y_valid_t = torch.Tensor(np.array(y_valid))

print(len(x_train))
print(len(x_valid))

train_dataset = torch.utils.data.TensorDataset(x_train_t, y_train_t)
test_dataset = torch.utils.data.TensorDataset(x_valid_t, y_valid_t)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(x_train),shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(x_valid),shuffle=True)

2310
578


In [38]:
# 训练 - LSC

use_gpu = True

if use_gpu:
    cnn = cnn.cuda()
else:
    cnn = cnn.cpu()

for epoch in range(1000):
    print('epoch {}'.format(epoch + 1))
    # training-----------------------------
    cnn.train()
    train_acc = 0.

    for step, (batch_x, batch_y) in enumerate(train_data_loader):
        batch_x, batch_y = Variable(batch_x), Variable(batch_y)
                
        if use_gpu:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            
#         print(batch_x.size())
        batch_x = batch_x.view(batch_x.size(0),1,1,-1)
            
        out = cnn(batch_x)
        loss = loss_function(out, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         print('Step:',step+1,'Finished! Loss:',loss.detach().cpu().numpy())
    print('Train Loss: {:.10f}'.format(loss))
#     print(classification_report(L_train_real,L_train_pred))

    # evaluation--------------------------------
    cnn.eval()
    with torch.no_grad():
        eval_acc = 0.

        for batch_x, batch_y in test_data_loader:
            batch_x, batch_y = Variable(batch_x), Variable(batch_y)

            if use_gpu:
                batch_x = batch_x.cuda()
                batch_y = batch_y.cuda()
            
            batch_x = batch_x.view(batch_x.size(0),1,1,-1)

            out = cnn(batch_x)
            loss = loss_function(out, batch_y)
            
        print('Valid Loss: {:.10f}'.format(loss))
#         print(classification_report(L_val_real,L_val_pred))

epoch 1
Train Loss: 1.3089696169
Valid Loss: 1.3061833382
epoch 2
Train Loss: 1.2251605988
Valid Loss: 1.2366032600
epoch 3
Train Loss: 1.1633152962
Valid Loss: 1.1902472973
epoch 4
Train Loss: 1.1212497950
Valid Loss: 1.1614679098
epoch 5
Train Loss: 1.0938686132
Valid Loss: 1.1429309845
epoch 6
Train Loss: 1.0748069286
Valid Loss: 1.1289762259
epoch 7
Train Loss: 1.0593920946
Valid Loss: 1.1166352034
epoch 8
Train Loss: 1.0453726053
Valid Loss: 1.1047599316
epoch 9
Train Loss: 1.0320177078
Valid Loss: 1.0932087898
epoch 10
Train Loss: 1.0193431377
Valid Loss: 1.0824332237
epoch 11
Train Loss: 1.0077978373
Valid Loss: 1.0730150938
epoch 12
Train Loss: 0.9978930950
Valid Loss: 1.0653361082
epoch 13
Train Loss: 0.9899243116
Valid Loss: 1.0594542027
epoch 14
Train Loss: 0.9838701487
Valid Loss: 1.0551265478
epoch 15
Train Loss: 0.9794284105
Valid Loss: 1.0519310236
epoch 16
Train Loss: 0.9761374593
Valid Loss: 1.0494278669
epoch 17
Train Loss: 0.9735325575
Valid Loss: 1.0472924709
epoch 

Valid Loss: 1.0301157236
epoch 144
Train Loss: 0.9526216984
Valid Loss: 1.0301125050
epoch 145
Train Loss: 0.9526184201
Valid Loss: 1.0301092863
epoch 146
Train Loss: 0.9526153207
Valid Loss: 1.0301061869
epoch 147
Train Loss: 0.9526121020
Valid Loss: 1.0301032066
epoch 148
Train Loss: 0.9526090026
Valid Loss: 1.0301003456
epoch 149
Train Loss: 0.9526059031
Valid Loss: 1.0300977230
epoch 150
Train Loss: 0.9526029229
Valid Loss: 1.0300949812
epoch 151
Train Loss: 0.9525999427
Valid Loss: 1.0300925970
epoch 152
Train Loss: 0.9525970221
Valid Loss: 1.0300900936
epoch 153
Train Loss: 0.9525940418
Valid Loss: 1.0300878286
epoch 154
Train Loss: 0.9525912404
Valid Loss: 1.0300854445
epoch 155
Train Loss: 0.9525883794
Valid Loss: 1.0300831795
epoch 156
Train Loss: 0.9525855184
Valid Loss: 1.0300807953
epoch 157
Train Loss: 0.9525827765
Valid Loss: 1.0300785303
epoch 158
Train Loss: 0.9525799751
Valid Loss: 1.0300762653
epoch 159
Train Loss: 0.9525772333
Valid Loss: 1.0300740004
epoch 160
Train

Train Loss: 0.9523906112
Valid Loss: 1.0299175978
epoch 283
Train Loss: 0.9523898363
Valid Loss: 1.0299170017
epoch 284
Train Loss: 0.9523891807
Valid Loss: 1.0299162865
epoch 285
Train Loss: 0.9523884654
Valid Loss: 1.0299158096
epoch 286
Train Loss: 0.9523876905
Valid Loss: 1.0299153328
epoch 287
Train Loss: 0.9523869753
Valid Loss: 1.0299146175
epoch 288
Train Loss: 0.9523863196
Valid Loss: 1.0299141407
epoch 289
Train Loss: 0.9523856640
Valid Loss: 1.0299135447
epoch 290
Train Loss: 0.9523848891
Valid Loss: 1.0299131870
epoch 291
Train Loss: 0.9523842335
Valid Loss: 1.0299125910
epoch 292
Train Loss: 0.9523835778
Valid Loss: 1.0299119949
epoch 293
Train Loss: 0.9523830414
Valid Loss: 1.0299113989
epoch 294
Train Loss: 0.9523822665
Valid Loss: 1.0299110413
epoch 295
Train Loss: 0.9523817301
Valid Loss: 1.0299105644
epoch 296
Train Loss: 0.9523809552
Valid Loss: 1.0299099684
epoch 297
Train Loss: 0.9523804188
Valid Loss: 1.0299094915
epoch 298
Train Loss: 0.9523798227
Valid Loss: 1.0

Train Loss: 0.9523421526
Valid Loss: 1.0298810005
epoch 422
Train Loss: 0.9523420930
Valid Loss: 1.0298810005
epoch 423
Train Loss: 0.9523419738
Valid Loss: 1.0298808813
epoch 424
Train Loss: 0.9523418546
Valid Loss: 1.0298808813
epoch 425
Train Loss: 0.9523417950
Valid Loss: 1.0298806429
epoch 426
Train Loss: 0.9523416162
Valid Loss: 1.0298806429
epoch 427
Train Loss: 0.9523414969
Valid Loss: 1.0298806429
epoch 428
Train Loss: 0.9523414373
Valid Loss: 1.0298806429
epoch 429
Train Loss: 0.9523414373
Valid Loss: 1.0298804045
epoch 430
Train Loss: 0.9523411989
Valid Loss: 1.0298804045
epoch 431
Train Loss: 0.9523411393
Valid Loss: 1.0298804045
epoch 432
Train Loss: 0.9523410201
Valid Loss: 1.0298802853
epoch 433
Train Loss: 0.9523409605
Valid Loss: 1.0298801661
epoch 434
Train Loss: 0.9523408413
Valid Loss: 1.0298801661
epoch 435
Train Loss: 0.9523407817
Valid Loss: 1.0298801661
epoch 436
Train Loss: 0.9523406625
Valid Loss: 1.0298800468
epoch 437
Train Loss: 0.9523405433
Valid Loss: 1.0

Valid Loss: 1.0298780203
epoch 561
Train Loss: 0.9523362517
Valid Loss: 1.0298780203
epoch 562
Train Loss: 0.9523362517
Valid Loss: 1.0298780203
epoch 563
Train Loss: 0.9523363709
Valid Loss: 1.0298780203
epoch 564
Train Loss: 0.9523362517
Valid Loss: 1.0298780203
epoch 565
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 566
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 567
Train Loss: 0.9523362517
Valid Loss: 1.0298780203
epoch 568
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 569
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 570
Train Loss: 0.9523362517
Valid Loss: 1.0298780203
epoch 571
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 572
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 573
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 574
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 575
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 576
Train Loss: 0.9523361921
Valid Loss: 1.0298780203
epoch 577
Train

Valid Loss: 1.0298781395
epoch 698
Train Loss: 0.9523359537
Valid Loss: 1.0298781395
epoch 699
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 700
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 701
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 702
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 703
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 704
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 705
Train Loss: 0.9523357749
Valid Loss: 1.0298781395
epoch 706
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 707
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 708
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 709
Train Loss: 0.9523358941
Valid Loss: 1.0298782587
epoch 710
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 711
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 712
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 713
Train Loss: 0.9523358941
Valid Loss: 1.0298781395
epoch 714
Train

Valid Loss: 1.0298782587
epoch 837
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 838
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 839
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 840
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 841
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 842
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 843
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 844
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 845
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 846
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 847
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 848
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 849
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 850
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 851
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 852
Train Loss: 0.9523357749
Valid Loss: 1.0298783779
epoch 853
Train

Valid Loss: 1.0298783779
epoch 975
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 976
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 977
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 978
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 979
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 980
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 981
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 982
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 983
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 984
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 985
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 986
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 987
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 988
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 989
Train Loss: 0.9523359537
Valid Loss: 1.0298783779
epoch 990
Train Loss: 0.9523358941
Valid Loss: 1.0298783779
epoch 991
Train

In [62]:
from sklearn import svm
from sklearn.metrics import mean_squared_error
clf = svm.SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

clf.fit(x_train,y_train)

# 运用原非平衡样本-评估
y_train_predict = clf.predict(x_train)
y_valid_predict = clf.predict(x_valid)

print('trainMSE:',mean_squared_error(y_train,y_train_predict))
# print(classification_report(y_train,y_train_predict)) #真实数据在前 训练结果在后！
print('validMSE:',mean_squared_error(y_valid,y_valid_predict))
# print(classification_report(y_test,y_test_predict))

trainMSE: 0.1122916433010682
validMSE: 0.09667053041486083


In [65]:
clf.fit(X,labels)

SVR(C=2.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [67]:
y_test_predict = clf.predict(test_data)

In [68]:
submit = pd.DataFrame(y_test_predict)

In [73]:
submit

,0
0,0.269911
1,0.243721
2,-0.091196
3,0.072408
4,0.246865
5,0.188211
6,-0.153986
7,0.185767
8,-0.097172
9,0.296530


In [74]:
submit.to_csv('C:/Users/jxjsj/Desktop/tianchi/steam/zhengqi_submit.txt',sep="\t",index = False) # bug!!!